In [1]:
library(lme4)
library(MCMCglmm)
library(broom)
library(nadiv)
library(tidyverse)

Loading required package: Matrix

Loading required package: coda

Loading required package: ape


Attaching package: 'nadiv'


The following object is masked from 'package:MCMCglmm':

    prunePed


-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.5     v dplyr   1.0.3
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x tidyr::expand() masks Matrix::expand()
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x tidyr::pack()   masks Matrix::pack()
x tidyr::unpack() masks Matrix::unpack()



In [2]:
#install.packages("nadiv")

In [3]:
library(parallel)
my.cores = detectCores()

# Download data

In [4]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
path_adapt = file.path(path_,'Adaptability')
path_model = file.path(path_adapt,'repeatability_model')
df = read.csv(file.path(path_adapt,'df_MVT_4individuality_withPCA.csv'), header = TRUE, sep = ",")
df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID) 
#mean-centering of the environmental variable so that intercepts reflect average values for the population and individuals
df$cDIB = as.integer(df$DIB) 
df$cDIB2 = poly(df$cDIB, degree=2,raw=TRUE)[,2]
df$RearingPenID = as.factor(df$RearingPenID) 
df$CLASS = as.factor(df$CLASS) 
df$Treatment = as.factor(df$Treatment) 
#scale all continuous fixed effectcs
df$temperature_C_avg_scale = scale(df$temperature_C_avg, center=TRUE, scale=TRUE)
df$cDIB2_scale = scale(df$cDIB2, center=TRUE, scale=TRUE)
df$cDIB_scale = scale(df$cDIB, center=TRUE, scale=TRUE)
df$avgDIB_scale = scale(df$avgDIB, center=TRUE, scale=TRUE)

df$PC1scale = scale(df$PC1, center=TRUE, scale=TRUE)

df = df %>%mutate(rel_severity = severity/mean(severity, na.rm=TRUE))
print(dim(df))
summary(df)

[1] 3230   42


      WIB            HenID           RPen            DIB          CLASS     
 Min.   :2.000   hen_116:  43   Min.   :1.000   Min.   :11.00   LEXP : 965  
 1st Qu.:4.000   hen_124:  43   1st Qu.:1.000   1st Qu.:23.00   MEXP :1379  
 Median :5.000   hen_127:  43   Median :2.000   Median :33.00   Other: 886  
 Mean   :5.222   hen_136:  43   Mean   :2.448   Mean   :33.59               
 3rd Qu.:7.000   hen_147:  43   3rd Qu.:3.000   3rd Qu.:44.00               
 Max.   :8.000   hen_151:  43   Max.   :4.000   Max.   :54.00               
                 (Other):2972                                               
             TrackingSystemID     PenID     temperature_C_avg Treatment  
 TrackingSystem 10-12:1302    pen10  :504   Min.   : 4.000    OFH :1614  
 TrackingSystem 3-5  :1069    pen11  :504   1st Qu.: 8.667    TRAN:1616  
 TrackingSystem 8-9  : 859    pen8   :430   Median :11.000               
                              pen9   :429   Mean   :10.838               
              

In [7]:
length(unique(df[['HenID']]))

[1] 80

In [8]:
nsamp <- 3230
burn <- 50000
thin <- 2000
(nitt <- burn + nsamp * thin)

[1] 6510000

# Random Slope Model

In [9]:
#prior
RR_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = diag(3), nu = 3, 
                                   alpha.mu = rep(0,3), 
                                   alpha.V = diag(3) * 25^2)))

In [10]:
#PC1
RR_model1 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale,
                     random = ~ us(1 + cDIB_scale + cDIB2_scale):HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model1)

#PC2
RR_model2 = MCMCglmm(PC2 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale,
                     random = ~ us(1 + cDIB_scale + cDIB2_scale):HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model2)

#PC3
RR_model3 = MCMCglmm(PC3 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale,
                     random = ~ us(1 + cDIB_scale + cDIB2_scale):HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model3)


                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteration = 19000

                       MCMC iteration = 20000

                       MC


 Iterations = 50001:6508001
 Thinning interval  = 2000
 Sample size  = 3230 

 DIC: 5109.719 

 G-structure:  ~us(1 + cDIB_scale + cDIB2_scale):HenID

                              post.mean l-95% CI u-95% CI eff.samp
(Intercept):(Intercept).HenID    0.4705  0.32481  0.63066     3230
cDIB_scale:(Intercept).HenID     0.1969 -0.03438  0.42488     3230
cDIB2_scale:(Intercept).HenID   -0.1440 -0.36450  0.07693     3230
(Intercept):cDIB_scale.HenID     0.1969 -0.03438  0.42488     3230
cDIB_scale:cDIB_scale.HenID      1.8225  1.22925  2.52957     3230
cDIB2_scale:cDIB_scale.HenID    -1.6292 -2.23963 -1.03904     3230
(Intercept):cDIB2_scale.HenID   -0.1440 -0.36450  0.07693     3230
cDIB_scale:cDIB2_scale.HenID    -1.6292 -2.23963 -1.03904     3230
cDIB2_scale:cDIB2_scale.HenID    1.6369  1.09698  2.26306     3230

 R-structure:  ~units

      post.mean l-95% CI u-95% CI eff.samp
units    0.2655   0.2523   0.2791     3230

 Location effects: PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + 


                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteration = 19000

                       MCMC iteration = 20000

                       MC


 Iterations = 50001:6508001
 Thinning interval  = 2000
 Sample size  = 3230 

 DIC: 6629.02 

 G-structure:  ~us(1 + cDIB_scale + cDIB2_scale):HenID

                              post.mean l-95% CI u-95% CI eff.samp
(Intercept):(Intercept).HenID   0.31312   0.2103   0.4204     3055
cDIB_scale:(Intercept).HenID   -0.12056  -0.3502   0.1074     3230
cDIB2_scale:(Intercept).HenID   0.01268  -0.1895   0.2112     2805
(Intercept):cDIB_scale.HenID   -0.12056  -0.3502   0.1074     3230
cDIB_scale:cDIB_scale.HenID     2.61795   1.7003   3.6180     3295
cDIB2_scale:cDIB_scale.HenID   -2.12092  -2.9793  -1.3579     3473
(Intercept):cDIB2_scale.HenID   0.01268  -0.1895   0.2112     2805
cDIB_scale:cDIB2_scale.HenID   -2.12092  -2.9793  -1.3579     3473
cDIB2_scale:cDIB2_scale.HenID   1.83634   1.1750   2.6093     3230

 R-structure:  ~units

      post.mean l-95% CI u-95% CI eff.samp
units    0.4265   0.4044   0.4481     3230

 Location effects: PC2 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + T


                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteration = 19000

                       MCMC iteration = 20000

                       MC


 Iterations = 50001:6508001
 Thinning interval  = 2000
 Sample size  = 3230 

 DIC: 6737.792 

 G-structure:  ~us(1 + cDIB_scale + cDIB2_scale):HenID

                              post.mean l-95% CI u-95% CI eff.samp
(Intercept):(Intercept).HenID    0.3847   0.2681   0.5161     3762
cDIB_scale:(Intercept).HenID     0.4746   0.2824   0.7032     3230
cDIB2_scale:(Intercept).HenID   -0.4921  -0.7223  -0.2829     3230
(Intercept):cDIB_scale.HenID     0.4746   0.2824   0.7032     3230
cDIB_scale:cDIB_scale.HenID      1.2744   0.7888   1.8427     3454
cDIB2_scale:cDIB_scale.HenID    -1.3313  -1.8759  -0.7882     3433
(Intercept):cDIB2_scale.HenID   -0.4921  -0.7223  -0.2829     3230
cDIB_scale:cDIB2_scale.HenID    -1.3313  -1.8759  -0.7882     3433
cDIB2_scale:cDIB2_scale.HenID    1.4874   0.9231   2.0818     3403

 R-structure:  ~units

      post.mean l-95% CI u-95% CI eff.samp
units    0.4432   0.4206   0.4653     3315

 Location effects: PC3 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + 

In [15]:
#posterior distribution of the among-individual variation in linear slopes term
mean(RR_model1$VCV[,"cDIB_scale:cDIB_scale.HenID"])
HPDinterval(RR_model1$VCV[,"cDIB_scale:cDIB_scale.HenID"])

#posterior distribution of the among-individual variation in quadratic slopes term
mean(RR_model1$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])
HPDinterval(RR_model1$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])

print('----------------correlation')
#################################### corelations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model1$VCV[,"cDIB_scale:(Intercept).HenID"]/
  (sqrt(RR_model1$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model1$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)

###correlation between intercept and quadratic linear slope###
corr_IQ = RR_model1$VCV[,"cDIB2_scale:(Intercept).HenID"]/
  (sqrt(RR_model1$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model1$VCV[,"cDIB2_scale:cDIB2_scale.HenID"]))
posterior.mode(corr_IQ)
HPDinterval(corr_IQ)

###correlation between linear slope and quadratic linear slope###
corr_LQ = RR_model1$VCV[,"cDIB_scale:cDIB2_scale.HenID"]/
  (sqrt(RR_model1$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])*
     sqrt(RR_model1$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_LQ)
HPDinterval(corr_LQ)

[1] 1.82246

,lower,upper
var1,1.229252,2.529573


[1] 1.63692

,lower,upper
var1,1.096978,2.263058


[1] "----------------correlation"


...

,lower,upper
var1,-0.03705448,0.4226086


var1 
-0.162346

,lower,upper
var1,-0.3928279,0.06946414


var1 
-0.9494624

,lower,upper
var1,-0.9677483,-0.9140633


In [16]:
#posterior distribution of the among-individual variation in linear slopes term
mean(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"])
HPDinterval(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"])

#posterior distribution of the among-individual variation in quadratic slopes term
mean(RR_model2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])
HPDinterval(RR_model2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])

print('----------------correlation')
#################################### corelations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model2$VCV[,"cDIB_scale:(Intercept).HenID"]/
  (sqrt(RR_model2$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)

###correlation between intercept and quadratic linear slope###
corr_IQ = RR_model2$VCV[,"cDIB2_scale:(Intercept).HenID"]/
  (sqrt(RR_model2$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"]))
posterior.mode(corr_IQ)
HPDinterval(corr_IQ)

###correlation between linear slope and quadratic linear slope###
corr_LQ = RR_model2$VCV[,"cDIB_scale:cDIB2_scale.HenID"]/
  (sqrt(RR_model2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])*
     sqrt(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_LQ)
HPDinterval(corr_LQ)

[1] 2.617945

,lower,upper
var1,1.700271,3.617975


[1] 1.836337

,lower,upper
var1,1.174955,2.609306


[1] "----------------correlation"


...

,lower,upper
var1,-0.3700683,0.1079584


var1 
0.01286481

,lower,upper
var1,-0.2521634,0.2523519


var1 
-0.9707992

,lower,upper
var1,-0.9831064,-0.949571


In [17]:
#posterior distribution of the among-individual variation in linear slopes term
mean(RR_model3$VCV[,"cDIB_scale:cDIB_scale.HenID"])
HPDinterval(RR_model3$VCV[,"cDIB_scale:cDIB_scale.HenID"])

#posterior distribution of the among-individual variation in quadratic slopes term
mean(RR_model3$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])
HPDinterval(RR_model3$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])

print('----------------correlation')
#################################### corelations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model3$VCV[,"cDIB_scale:(Intercept).HenID"]/
  (sqrt(RR_model3$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model3$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)

###correlation between intercept and quadratic linear slope###
corr_IQ = RR_model3$VCV[,"cDIB2_scale:(Intercept).HenID"]/
  (sqrt(RR_model3$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model3$VCV[,"cDIB2_scale:cDIB2_scale.HenID"]))
posterior.mode(corr_IQ)
HPDinterval(corr_IQ)

###correlation between linear slope and quadratic linear slope###
corr_LQ = RR_model3$VCV[,"cDIB_scale:cDIB2_scale.HenID"]/
  (sqrt(RR_model3$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])*
     sqrt(RR_model3$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_LQ)
HPDinterval(corr_LQ)

[1] 1.274368

,lower,upper
var1,0.7887545,1.84269


[1] 1.487354

,lower,upper
var1,0.923114,2.081832


[1] "----------------correlation"


...

,lower,upper
var1,0.5211626,0.8320886


var1 
-0.6831185

,lower,upper
var1,-0.801987,-0.4886584


var1 
-0.9679067

,lower,upper
var1,-0.9832064,-0.94763
